In [ ]:
import temgymbasic.jax_components as comp
from temgymbasic.jax_run import run_to_end
from temgymbasic.jax_ray import Ray
from temgymbasic.jax_source import ParallelBeam
import jax.numpy as jnp
import jax
import sympy as sp
# from .utils import circular_beam, point_beam

In [ ]:
model = [comp.Lens(z= 0.5, focal_length=0.1),
         comp.DoubleDeflector(z=0.325,
                             first=comp.Deflector(z=0.3, def_x=0.1, def_y=0.0), 
                             second=comp.Deflector(z=0.35, def_x=0.1, def_y=0.0)),
         comp.Biprism(z=0.75, deflection=0.001),
         comp.Aperture(z=0.8, radius=0.01),
         comp.Detector(z=2.0, pixel_size=0.01, shape=(100, 100))]

n_rays = 1
n_rays_dim = int(jnp.sqrt(n_rays))

input_amplitude = 1.0
input_pathlength = 0.0
input_wavelength = 1.0

# Prepare multiple input rays (each row is a separate ray)
r1mx = jnp.linspace(-0.1, 0.1, n_rays_dim) #x off set
r1my = jnp.linspace(-0.1, 0.1, n_rays_dim) #y off set
r1m = jnp.stack(jnp.meshgrid(r1mx, r1my), axis=-1).reshape(2, -1)

theta1mx = jnp.linspace(-0.1, 0.1, n_rays_dim) #dx off set
theta1my = jnp.linspace(-0.1, 0.1, n_rays_dim) #dy off set
theta1m = jnp.stack(jnp.meshgrid(theta1mx, theta1my), axis=-1).reshape(2, -1)

#Create ray input
input_z = jnp.zeros((n_rays))
input_matrix = jnp.vstack([r1m[0, :], r1m[1, :], theta1m[0, :], theta1m[1, :], jnp.ones((n_rays))]).T
input_amplitude = jnp.ones((n_rays))
input_pathlength = jnp.zeros((n_rays))
input_wavelength = jnp.ones((n_rays))

Rays = Ray(z=input_z, 
           matrix=input_matrix, 
           amplitude=input_amplitude, 
           pathlength=input_pathlength, 
           wavelength=input_wavelength)


def create_ray(ray):
    return Ray(z=ray.z, 
               matrix=ray.matrix,
               amplitude=ray.amplitude,
               pathlength=ray.pathlength,
               wavelength=ray.wavelength)

# vmap over run_model to process each ray in rays_input

# Method 1
batched_run_model_one_ray = jax.jit(jax.vmap(lambda r: run_to_end(create_ray(r), model)))
all_rays1 = batched_run_model_one_ray(Rays)

# Method 2
batched_run_model_array_of_rays = jax.jit(jax.vmap(run_to_end, in_axes=(0, None)))
all_rays2 = batched_run_model_array_of_rays(Rays, model)

# Method 3
all_rays3 = jax.jit(run_to_end)(Ray(z=0.0, 
                                         matrix=input_matrix, 
                                         amplitude=1., 
                                         pathlength=0., 
                                         wavelength=1.),
                                         model)

In [3]:
# Test the the output ray matrix of all 3 methods is the same
print(jnp.allclose(all_rays1, all_rays2))
print(jnp.allclose(all_rays2, all_rays3))
print(jnp.allclose(all_rays1, all_rays3))

True
True
True


In [4]:
Rays = ParallelBeam(z=0,
                    radius=0.1,
                    n_rays=1024,
                    random=False,
                    voltage=1.0)
                    
batched_run_model_one_ray(Rays)

\\grenfshome01\DL277493\TemGym\src\temgymbasic\jax_utils.py:166: UserWarning: Explicitly requested dtype <class 'jax.numpy.float64'> requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  matrix = jnp.zeros(


Array([[-0.04117647, -0.07131974, -0.02941176, -0.05094267,  1.        ],
       [-0.04117647, -0.07131974, -0.02941176, -0.05094267,  1.        ],
       [-0.04117647, -0.07131974, -0.02941176, -0.05094267,  1.        ],
       ...,
       [-1.3978741 , -0.07712287, -0.9984815 , -0.05508776,  1.        ],
       [-1.3978741 , -0.07712287, -0.9984815 , -0.05508776,  1.        ],
       [-1.3978741 , -0.07712287, -0.9984815 , -0.05508776,  1.        ]],      dtype=float32)

In [5]:
%timeit batched_run_model_array_of_rays(Rays, model)

23.7 μs ± 276 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [6]:
func = jax.jit(run_to_end)
%timeit func(Ray(z=0.0, matrix=input_matrix, amplitude=1., pathlength=0., wavelength=1.), model)

22.7 μs ± 308 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
# Method 1
batched_dro_dri = jax.vmap(lambda ray: jax.jacobian(run_to_end, argnums=0)(ray, model))
dro_dri = batched_dro_dri(Rays)

# # Method 2
batched_run_model_array_of_rays = jax.vmap(jax.jacobian(run_to_end), in_axes=(0, None))
dro_dri = batched_run_model_array_of_rays(Rays, model)


# Method 3
dro_dri = jax.jacobian(run_to_end, argnums=0)(Ray(z=0.0, matrix=input_matrix, amplitude=1., pathlength=0., wavelength=1.), model)


In [8]:
ABCD = dro_dri.matrix.squeeze()
print(ABCD)
# Extract ABCD matrices
# A = ABCD[:, :2, :2]
# B = ABCD[:, :2, 2:4]
# C = ABCD[:, 2:4, :2]
# D = ABCD[:, 2:4, 2:4]


[[-14.    0.   -5.5   0.    0. ]
 [  0.  -14.    0.   -5.5   0. ]
 [-10.    0.   -4.    0.    0. ]
 [  0.  -10.    0.   -4.    0. ]
 [  0.    0.    0.    0.    0. ]]


In [9]:

# batched_run_model = jax.vmap(lambda ray: run_model_iter(ray, model))
# all_rays = batched_run_model(Rays)

# # # Partial derivatives of ray output with respect to ray input
# batched_dro_dri = jax.vmap(lambda ray: jax.jacobian(run_model_to_end, argnums=0)(ray, model))
# dro_dri = batched_dro_dri(Rays)

# ABCD = dro_dri.matrix.matrix

# # Extract ABCD matrices
# A = ABCD[:, :2, :2]
# B = ABCD[:, :2, 2:4]
# C = ABCD[:, 2:4, :2]
# D = ABCD[:, 2:4, 2:4]



In [10]:
focal_length = model[0].focal_length
expected_C = -1 / focal_length * jnp.eye(2)
expected_C = jnp.repeat(expected_C[None, :, :], C.shape[0], axis=0)

# Verify that the 2x2 matric C is equal to -1/f on each diagonal element
assert jnp.allclose(C, expected_C), "C does not match -1/f * I"
print("Test passed: C is equal to -1/f * I.")

NameError: name 'C' is not defined

In [ ]:
# # Partial derivatives of ray output with respect to model parameters
batched_dro_dri = jax.jit(jax.vmap(lambda r: jax.jacobian(run_model_to_end, argnums=1)(create_ray(r), model)))
dro_dmodel = batched_dro_dri(rays_input)

print(dro_dmodel)
